<center>
<h1>Workshop 2: Quality Control (QC)</h1><br><br>
<i><big>Preparing and filtering data for genome-wide association studies.</big><br><br>
Hatzikotoulas Konstantinos (Kostas) (hatzikotoulas@helmholtz-muenchen.de)</i>
</center>

## Objectives

In this workshop, you will learn the data quality assessment and control steps that are typically carried out during genome wide association studies (GWAS).

## Why do we need the QC?

Study design and errors in genotype calling can introduce systematic biases into GWAS, leading to spurious associations.
A thorough QC can help us identify samples and markers that should be removed prior to association analysis in order to minimize the number of false-positive and false-negative associations. 

<img src="FN.png" width="40%">

In this practical, we assume that the study design has been conducted appropriately and the QC applies to genotypes after they have been called from probe intensity data.

## QC protocol

The QC protocol of a GWAS is usually split into two broad categories, “Sample QC” and “Variant QC”.
Sample QC is done prior to Variant QC because we want to maximise the number of markers remaining in the study.

We will be using <code>plink</code> to run the QC and R to visualise the results.
You can find a manual and command reference of <code>plink</code> [here](https://www.cog-genomics.org/plink/1.9/) and [here](https://www.cog-genomics.org/plink/1.9/index).

## Sample QC

It consists of (at least) five steps:

<img src="QCsteps.png" width="40%">



<br>
Initial setup:
<br>

<br>

<div class="alert alert-warning">
<b>Location:</b> The data for this workshop can be found in this directory: <code>/dss/dsshome1/lxc06/gobi012/GWAS/gobi/data/Workshop2_QC/</code>. 

</div>

...but for this workshop we have created a container with all files/software needed


<br>
Initial setup:
<br>

```bash
# login
module load charliecloud
ch-run /dss/dsshome1/lxc06/gobi012/GWAS/gobi -- ls -lrt --color=auto /data
ch-run /dss/dsshome1/lxc06/gobi012/GWAS/gobi -- /bin/bash
## test plink runs in the container:
plink --help
## test that R runs for you, then use q() and then n to exit R
R

#Once in the container move to your home directory 
cd ~

#Make your own directory and give the path of your DIR and initial unqced FILE
mkdir GWASgobi[group number]_QC_[your name]
cd GWASgobi[group number]_QC_[your name]
(eg mkdir GWASgobi012_QC_Kostas)

#Copy the initial UnQCed data to your present working directory (pwd)

scp -pr /data/Workshop2_QC/* ./

DIR=<path to your directory>

(eg DIR=/home/gobi012/GWASgobi012_QC_Kostas)

FILE=VSS


```

For each quesiton please provide the following in a text file or as markdown in the Jupyter note book:
- Answers
- Any code generated
For any plots please export them as .png, .jpg or .pdf and label them as GWAS.[question number].

Each question answered correctly will be awarded 0.89 point. 25 points in total for practical 2


#### In the 5 first steps we will generate all of the files required by using plink and then we will visualise the results in R.

### Step_1: Individuals with outlying missing genotype

#### Call rate (CR)

<div class="alert alert-success"><b>Question 1a:</b> Run missingness across file genome-wide </div>

<div class="alert alert-info" role="alert"><b>Answer 1a:</b> 


    
plink --bfile $DIR/$FILE --missing --out $DIR/$FILE-missing 
(this is an example of what we expect to see as an answer)


</div> 

<div class="alert alert-success"><b>Question 1b:</b> Use plink to exclude samples with a call rate less than 0.98 (use VSS.bed/bim/fam) </div>


<div class="alert alert-info" role="alert"><b>Answer 1b:</b>
plink --bfile $DIR/$FILE --mind 0.02 --make-bed --out $VSS-mind

 </div>

### Step_2: Individuals with discordant sex information

#### Sex check

<div class="alert alert-success"><b>Question 2a:</b> Run sex checking (use VSS.bed/bim/fam) </div>

<div class="alert alert-info" role="alert"><b>Answer 2a:</b>
plink --bfile $DIR/$FILE --check-sex --make-bed --out $FILE-checksex
</div>





<div class="alert alert-success"><b>Question 2b:</b> Extract Xchr SNPs (use VSS.bed/bim/fam) </div>

<div class="alert alert-info" role="alert"><b>Answer 2b:</b>

plink --bfile $DIR/$FILE --chr X --make-bed --out $FILE-xchrsnps

</div>



<div class="alert alert-success"><b>Question 2c:</b> Run missingness on Xchr SNPs </div>

<div class="alert alert-info" role="alert"><b>Answer 2c:</b> 

plink --bfile $DIR/$FILE-xchrsnps --missing --make-bed --out $FILE-xchrsnps-missing

</div>



### Step_3: Individuals with outlying heterozygosity rate

#### Heterozygosity

<div class="alert alert-success"><b>Question 3a:</b> Produce a file containing only the autosomal variants (use VSS.bed/bim/fam) </div>

<div class="alert alert-info" role="alert"><b>Answer 3a:</b>

plink --bfile $DIR/$FILE --autosome --make-bed --out $FILE-autosomal

</div>



We need to split the dataset into rare/low frequency and common minor allele frequency (MAF) and look at the missingness separately for each

<div class="alert alert-success"><b>Question 3b:</b> Extract autosomal SNPs with MAF greater than/equal to 1% </div>

<div class="alert alert-info" role="alert"><b>Answer 3b:</b> 

plink --bfile $DIR/$FILE-autosomal --maf 0.01 --het --make-bed -out $FILE-mafgre1p

</div>



<div class="alert alert-success"><b>Question 3c:</b> Extract autosomal SNPs with MAF less than 1% </div>

<div class="alert alert-info" role="alert"><b>Answer 3c:</b> 

plink --bfile $DIR/$FILE-autosomal --max-maf 0.01 --het --make-bed --out $FILE-mafles1p

</div>



<div class="alert alert-success"><b>Question 3d:</b> Get missingness of each MAF file (eg greater than/equal to 1% and less than 1%) to plot against heterozygosity in R
</div>





<div class="alert alert-info" role="alert"><b>Answer 3d:</b>

plink --bfile $DIR/$FILE-mafles1p --missing --make-bed --out $FILE-mafles1p-miss

plink --bfile $DIR/$FILE-mafgre1p --missing --make-bed -out $FILE-mafgre1p-miss
</div>

### Step_4: Duplicated or related individuals

#### Relatedness/Duplicates

Pair-wise IBD to look at duplicates.A) Using only variants ≥1%, excluding complex regions and B) LD prune using R-squared 0.2, a window size of 50 variants and 5 variants to shift the window at the end of each step

<div class="alert alert-success"><b>Question 4a:</b> Use autosomal SNPs with MAF ≥1% and exclude complex regions (please use this file: complex_regions.txt)</div>

<div class="alert alert-info" role="alert"><b>Answer 4a:</b> 

plink --bfile $DIR/$FILE-mafgre1p --exclude complex_regions.txt --make-bed --out $FILE-excludecomplex

    
</div>




<div class="alert alert-success"><b>Question 4b:</b> LD prune </div>

<div class="alert alert-info" role="alert"><b>Answer 4b:</b>


plink --bfile $DIR/$FILE-mafgre1p --indep-pairwise 50['kb'] 5 0.2 --make-bed --out $FILE-LDprune


</div>



<div class="alert alert-success"><b>Question 4c:</b> Calculate pair-wise IBD </div>



<div class="alert alert-info" role="alert"><b>Answer 4c:</b>

plink --bfile $DIR/$FILE-mafgre1p --genome --make-bed --out $FILE-ibd


</div>




### Step_5: Ethnicity outliers

#### Ethnicity MDS distance matrix

For this step, you will need to merge your data with 1000 Genomes genotype data (or HapMap genotype data).
(Its a time consuming procedure thus please use the following file)



```

FILE2=VSS-1Kg

```

##### Pair-wise IBD.

<div class="alert alert-success"><b>Question 5a:</b> Use autosomal SNPs with MAF ≥1% and exclude complex regions. </div>

<div class="alert alert-info" role="alert"><b>Answer 5a:</b> 

plink --bfile $DIR/$FILE2 --maf --exclude complex_regions.txt --make-bed --out $FILE2-gre1p-exclucomplex


</div>





<div class="alert alert-success"><b>Question 5b:</b> LD prune using R-squared 0.2, a window size of 50 variants and 5 variants to shift the window at the end of each step. </div>

<div class="alert alert-info" role="alert"><b>Answer 5b:</b>

plink --bfile $DIR/$FILE2-gre1p-exclucomplex --indep-pairwise 50['kb'] 5 0.2 --make-bed --out $FILE2-LDprune

</div>



<div class="alert alert-success"><b>Question 5c:</b> Calclulate pair-wise IBD </div>

<div class="alert alert-info" role="alert"><b>Answer 5c:</b> 

plink --bfile $DIR/$FILE2-LDprune --genome --make-bed --out $FILE2-ibd

</div>




<div class="alert alert-success"><b>Question 5d:</b> MDS distance matrix calcualting the first 10 components </div>

<div class="alert alert-info" role="alert"><b>Answer 5d:</b> 


plink --bfile  $DIR/$FILE2-ibd --pca 10 --make-bed --out $FILE2-pca10

</div>



### Now, we need to check our results against R plots

%%R

##### -----------------------------------------------------------------------------------------------------------------#
#####         (6)  SAMPLE CALL RATE    - threshold = 98%                                                               #
##### -----------------------------------------------------------------------------------------------------------------#


<div class="alert alert-success"><b>Question 6a:</b> Plot the missingness of the individuals (eg a histogram of xlab=missingness and ylab=frequency) </div>
<div class="alert alert-success"><b>Question 6b:</b> Create a .txt file with the samples to be removed (missingness of >2%) </div>

In [ ]:
#6a
missingData <- read.table(VSS-missing.imiss)
hist(missingData$F_MISS, xlab = 'missingness', ylab='frequency', main = "Missingness of Individuals")

In [ ]:
#6b
removedData <- read.table(VSS-mind.irem)
write.table(removedData, file = "VSS-mind.txt", sep = "\t", row.names = FALSE, col.names = FALSE)

##### -----------------------------------------------------------------------------------------------------------------#
#####          (7) SEX CHECKING                                                                                         #
##### -----------------------------------------------------------------------------------------------------------------#
<div class="alert alert-success"><b>Question 7a:</b> Plot the sex-check results (eg xlab="X chr inbreeding (homozygosity) estimate F", ylab="Proportion of missing SNPs for the X chr") </div>
<div class="alert alert-success"><b>Question 7b:</b> Create a .txt file with the samples to be removed (status:problem) </div>

In [ ]:
#7a
checksexData <- read.table("VSS-checksex.sexcheck", h=T)
xchrsnpsMissData <- read.table("VSS-xchrsnps-missing.imiss",h=T)

sex_xchrmData <- merge(checksexData,xchrsnpsMissData, by=c("FID","IID"))
plot(sex_xchrmData$F,sex_xchrmData$N_MISS,xlab="X chr inbreeding (homozygosity) estimate F", 
     ylab="Proportion of missing SNPs for the X chr")


In [ ]:
#7b
sex-problemData <- subset(sex_xchrmData, STATUS = "PROBLEM", select = IID)
write.table(sex_problemData, file = "sex-problem.txt",sep ="\t", row.names = FALSE, col.names = FALSE)


##### -----------------------------------------------------------------------------------------------------------------#
#####      (8)    Heterozygosity   MAF≥1% and MAF<1%                                                                   #
##### -----------------------------------------------------------------------------------------------------------------#

<div class="alert alert-success"><b>Question 8a:</b> Plot the heterozygosity results (maf>=1%) against missingness (eg xlab="% Heterozygosity", ylab="F_MISS") </div>
<div class="alert alert-success"><b>Question 8b:</b> Create a .txt file with the samples to be removed (more than 3xSD away from the mean) </div>
<div class="alert alert-success"><b>Question 8c:</b> Repeat the above steps for the heterozygosity results of maf less than 1% </div>

In [1]:
#8a
mafg1miss <- read.table("VSS-mafgre1p-miss.imiss", h =T)
mafg1het <- read.table("VSS-mafgre1p.het", h = T)
g1hemiss <- merge(mafg1miss, mafg1het, by = c("FID","IID"))

names(g1hemiss)[7] <- "O"
names(g1hemiss)[9] <- "N"

g1hemiss$O  <- as.numeric(g1hemiss$O)
g1hemiss$N  <- as.numeric(g1hemiss$N)

plot(g1hemiss$O/g1hemiss$N, g1hemiss$F_MISS,xlab="% Heterozygosity", ylab="F_MISS", main ="Heterozygosity MAF≥1%" )

NameError: name 'mafg1' is not defined

In [ ]:
#8b
g1hemiss["F_MISS"] <- lapply(g1hemiss["F_MISS"],
      function(x) replace(x,abs(scale(x))>3,NA))

SD3_removed <-  subset(g1hemiss, is.na(F_MISS), select = c("FID","IID"))
write.table(SD3_removed, file = "gre1_SD3_removed.txt", sep = "\t", row.names = FALSE, col.names = TRUE)

In [ ]:
#8c
mafles1miss <- read.table("VSS-mafles1p-miss.imiss",h= T)

mafles1het <- read.table("VSS-mafles1p.het", h=T)

les1hemiss <- merge(mafles1miss, mafles1het, by = c("FID","IID"))


names(les1hemiss)[7] <- "O"
names(les1hemiss)[9] <- "N"

les1hemiss$O  <- as.numeric(les1hemiss$O)
les1hemiss$N  <- as.numeric(les1hemiss$N)

plot(les1hemiss$O/les1hemiss$N, les1hemiss$F_MISS,xlab="% Heterozygosity", ylab="F_MISS", main = "Heterozygosity MAF<1%" )

In [ ]:
#8c
les1hemiss["F_MISS"] <- lapply(les1hemiss["F_MISS"],
      function(x) replace(x,abs(scale(x))>3,NA))

les1_SD3_removed <-  subset(les1hemiss, is.na(F_MISS), select = c("FID","IID"))
write.table(les1_SD3_removed, file = "les1_SD3_removed.txt", sep = "\t", row.names = FALSE, col.names = TRUE)

##### -----------------------------------------------------------------------------------------------------------------#
#####      (9)   DUPLICATES - RELATEDNESS                                  
##### -----------------------------------------------------------------------------------------------------------------#

<div class="alert alert-success"><b>Question 9a:</b> Plot the IBD results (eg PI_HAT) </div>
<div class="alert alert-success"><b>Question 9b:</b> Create 2 .txt files with the samples to be removed. One for duplicates/twins and one for related individuals of a PI_HAT value >0.2 </div>


In [ ]:
#9a
ibd_result <- read.table("VSS-ibd.genome",h = T)
hist(ibd_result$PI_HAT)

In [ ]:
#9b
identicalData <- subset(ibd_result, PI_HAT > 0.9, select = c(IID1,IID2,Z2))
write.table(identicalData, file = "VSS-identical.txt", sep = "\t", row.names = FALSE, col.names = TRUE)

relativData <-  subset(ibd_result, PI_HAT > 0.2, select = c(IID1,IID2,PI_HAT))
write.table(relativData, file = "VSS-relative.txt", sep = "\t", row.names = FALSE, col.names = TRUE)

##### -----------------------------------------------------------------------------------------------------------------#
#####      (10)   ETHNICITY           Merged with 1000 genomes file                                                     #
##### -----------------------------------------------------------------------------------------------------------------#

#### The Populationfile-1kg.txt contains individuals' ethnicities

<div class="alert alert-success"><b>Question 10a:</b> Merge the population information (Populationfile-1kg.txt) into the genome data and plot PC1 and PC2 - subsetting by population (eg use different coulours) </div>
<div class="alert alert-success"><b>Question 10b:</b> Identify the outliers and create a .txt file  </div>
<div class="alert alert-success"><b>Question 10c:</b> Remove the outliers from the population file and plot again </div>


In [ ]:
#10a
population_data <- read.table("Populationfile-1kg.txt",h =T)
vss1kgibd <- read.table("VSS-1Kg-pca10.eigenvet")

names(vss1kgpca)[2] <- "ID"

pca_pop_data <- merge(population_data,vss1kgpca, by(ID))

plot(pca_pop_data$V3,pca_pop_data$V4, col = "Population")



ggplot(pca_pop_data,aes(V3,V4, color = Population))+geom_point()+
labs(title ="PC1 and PC2 - subsetting by population", x = "PC1", y = "PC2")


In [ ]:
#10b
population_data <- read.table("Populationfile-1kg.txt",h =T)
vss1kgpca <- read.table("VSS-1Kg-pca10.eigenvec")

names(vss1kgpca)[2] <- "ID"

pca_pop_data <- merge(population_data,vss1kgpca, by = "ID")


pca_pop_data["V3"] <- lapply(pca_pop_data["V3"],
      function(x) replace(x,abs(scale(x))>3,NA))

pca_pop_data["V4"] <- lapply(pca_pop_data["V4"],
      function(x) replace(x,abs(scale(x))>3,NA))


outliers <- subset(pca_pop_data, (is.na(V3)|is.na(V4)), select = c("ID","V1"))


write.table(outliers , file = "PCA_outliers.txt", sep = "\t", row.names = FALSE, col.names = TRUE)

In [ ]:
#10c
ggplot(pca_pop_data2,aes(V3,V4, color = Population))+
geom_point()+
labs(title ="PC1 and PC2 - subsetting by population", x = "PC1", y = "PC2")

ggplot(U_id,aes(V3,V4, color = Population))+
geom_point()+
labs(title ="PC1 and PC2 - subsetting by population", x = "PC1", y = "PC2")

#### Combine all outliers' files and exclude them from your data

## Variant QC

It consists of (at least) four steps:

    1.Identification of variants with an excessive missing genotype
    2.Identification of variants demonstrating a significant deviation from Hardy-Weinberg equilibrium (HWE)
    3.Removal of all makers with a very low minor allele frequency
    4.Removal of all makers with cluster separation score <0.4

The criteria used to filter out low quality markers differ from study to study.
Variant QC must be done with great care as every removed marker is potentially a missed disease variant.
Imputation can be used to recover some of the excluded markers

Here we are using the following thresholds:

- Call rate = 98%
- HWE       = p <=1x10-4
- MAF       = 1%
- Cluster separation score = <=0.4

<div class="alert alert-success"><b>Question 11a:</b> Run the plink command for all these thresholds. The cluster separation scores are provided to you (coreex_argogs_20141015.clustersep.txt) </div>



<div class="alert alert-info" role="alert"><b>Answer 11a:</b>
plink --bfile  $DIR/$FILE --geno 0.02 --hwe 1e-4 "midp" --maf 0.01 --exclude coreex_argogs_20141015.clustersep.txt --make-bed --out $FILE-variantqc

</div>

